In [3]:
import pandas as pd

In [4]:
import os
import sys

print(os.getcwd())

/Users/jacklm/Documents/UNC/POLI 490H/recidivism


In [5]:
df = pd.read_csv('./data/example_arrests.csv')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (18,19,22,52,241) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
import numpy as np
df.replace({'.':np.nan}, inplace=True)
df.dropna(axis=1, inplace=True, thresh=5000)
df.columns

Index(['CRRKCY', 'CRRKCC', 'CRRKYY', 'CRRKSQ', 'CRRTYP', 'CRRNAM', 'CRRADD',
       'CRRCTY', 'CRRDST', 'CRRZIP', 'CRRDOB', 'CRRACE', 'CRRSEX', 'CRRSIL',
       'CRRPRC', 'CRRWNO', 'CRRIDT', 'CRRRCD', 'CRRDTS', 'CRRTDT', 'CRRCRT',
       'CRRCDL', 'CRRCMV', 'CRRHAZ', 'CRRLUP', 'CRRCAD', 'CRRCAS', 'CRRCAJ',
       'CRRCAN', 'CRRVTY', 'FILLER3', 'CRRDAT', 'CRRDDT', 'CRRDDI', 'CRRDSF',
       'CRRPRS', 'CRRBONDT', 'CRRBONDA', 'CRRVLN', 'CRRVLS', 'CRRCHR',
       'CRRCMN', 'CRRCAP', 'CRRVID', 'CRRCRDT', 'CRRPROCI', 'CRRCSE', 'LN01',
       'SEGSEQ_offense', 'LN02', 'RECID', 'CROLNO', 'CROFFC', 'CROCDT',
       'CROFFA', 'CRDMOD', 'CRDDDT', 'CRDCDMVL', 'CRDLTDDP', 'crrkey',
       'CRNADTAT', 'CRNAMAAT', 'CRNDATAT', 'CRNADTDA', 'CRNINTDA', 'CRNAMADA',
       'CRNDATDA'],
      dtype='object')

In [7]:
df['CRRSEX'].hist()

In [0]:
df.hist()